In [1]:
# imports

import os
import logging
from dotenv import load_dotenv
from huggingface_hub import login
import numpy as np
import re
from sentence_transformers import SentenceTransformer
import chromadb
from sklearn.manifold import TSNE
import plotly.graph_objects as go
from litellm import completion
from tqdm.notebook import tqdm
from agents.evaluator import evaluate
from agents.items import Item

In [2]:
# environment

load_dotenv(override=True)
DB = "products_vectorstore"

In [3]:
# Log in to HuggingFace
# If you don't have a HuggingFace account, you can set one up for free at www.huggingface.co
# And then add the HF_TOKEN to your .env file as explained in the project README

hf_token = os.environ['HF_TOKEN']
login(token=hf_token, add_to_git_credential=False)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [4]:
LITE_MODE = True

In [5]:
username = "ed-donner"
dataset = f"{username}/items_lite" if LITE_MODE else f"{username}/items_full"

train, val, test = Item.from_hub(dataset)

print(f"Loaded {len(train):,} training items, {len(val):,} validation items, {len(test):,} test items")

Loaded 20,000 training items, 1,000 validation items, 1,000 test items


# Now create a Chroma Datastore

Now we will use the free, open-source Vector database Chroma.  
We will create a Chroma datastore with 400,000 products from our training dataset.

In [6]:
client = chromadb.PersistentClient(path=DB)

In [7]:
encoder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [8]:
# Pass in a list of texts, get back a numpy array of vectors

vector = encoder.encode(["A proficient AI engineer who has almost reached the finale of AI Engineering Core Track!"])[0]
print(vector.shape)
vector

(384,)


array([-5.68233877e-02, -6.70465603e-02,  4.41129468e-02,  5.98605815e-03,
       -2.28948686e-02, -2.95300744e-02,  5.56368902e-02,  3.42665389e-02,
       -1.08529337e-01, -3.81690711e-02, -7.43872374e-02, -1.03664294e-01,
        1.69147886e-02,  1.33929669e-03, -6.86191097e-02,  8.99353623e-02,
       -1.45186232e-02, -2.43885256e-02,  4.21856344e-03, -9.62992534e-02,
       -2.51799170e-02,  4.60675918e-02,  4.95348172e-03, -3.88680138e-02,
        1.07718771e-03,  6.82337061e-02, -1.13859950e-02, -5.83416522e-02,
       -1.03801191e-02, -1.74953155e-02, -1.86478309e-02,  4.07057302e-03,
        1.59438103e-02,  6.49722964e-02,  3.71176079e-02,  2.78225057e-02,
       -4.41945046e-02, -2.34372858e-02,  9.71035883e-02, -5.06138988e-02,
       -1.93864573e-02, -3.83471660e-02,  4.76066805e-02, -3.36107425e-02,
        5.08287288e-02,  3.57934833e-02,  2.91817449e-03, -1.06529139e-01,
        4.07211669e-02, -5.85432805e-04, -1.05607428e-01, -1.03584342e-01,
        3.71123739e-02, -

In [9]:
# Check if the collection exists; if not, create it

collection_name = "products"
existing_collection_names = [collection.name for collection in client.list_collections()]

if collection_name not in existing_collection_names:
    collection = client.create_collection(collection_name)
    for i in tqdm(range(0, len(train), 1000)):
        documents = [item.summary for item in train[i: i+1000]]
        vectors = encoder.encode(documents).astype(float).tolist()
        metadatas = [{"category": item.category, "price": item.price} for item in train[i: i+1000]]
        ids = [f"doc_{j}" for j in range(i, i+1000)]
        ids = ids[:len(documents)]
        collection.add(ids=ids, documents=documents, embeddings=vectors, metadatas=metadatas)

collection = client.get_or_create_collection(collection_name)

# Let's visualize the vectorized data

In [10]:
# It is very fun turning this up to 800_000 and seeing the full dataset visualized,
# but it almost crashes my box every time so do that at your own risk!! 10_000 is safe!

MAXIMUM_DATAPOINTS = 10_000

In [11]:
CATEGORIES = ['Appliances', 'Automotive', 'Cell_Phones_and_Accessories', 'Electronics','Musical_Instruments', 'Office_Products', 'Tools_and_Home_Improvement', 'Toys_and_Games']
COLORS = ['cyan', 'blue', 'brown', 'orange', 'yellow', 'green' , 'purple', 'red']

In [12]:
# Prework
result = collection.get(include=['embeddings', 'documents', 'metadatas'], limit=MAXIMUM_DATAPOINTS)
vectors = np.array(result['embeddings'])
documents = result['documents']
categories = [metadata['category'] for metadata in result['metadatas']]
colors = [COLORS[CATEGORIES.index(c)] for c in categories]

In [13]:
# Let's try a 2D chart
# TSNE stands for t-distributed Stochastic Neighbor Embedding - it's a common technique for reducing dimensionality of data

tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

In [14]:
# Create the 2D scatter plot
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=4, color=colors, opacity=0.7),
    text=[f"Category: {c}<br>Text: {d[:50]}..." for c, d in zip(categories, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='2D Chroma Vectorstore Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y'),
    width=1200,
    height=800,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

In [15]:
# Let's try 3D!

tsne = TSNE(n_components=3, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

In [16]:
# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=2, color=colors, opacity=0.7),
    text=[f"Category: {c}<br>Text: {d[:50]}..." for c, d in zip(categories, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='3D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
    width=1200,
    height=800,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

In [17]:
test[0]

<Old Blood Noise Excess V2 Distortion Chorus/Delay Pedal = $219.0>

In [18]:
def vector(item):
    return encoder.encode(item.summary)

In [19]:
def find_similars(item):
    vec = vector(item)
    results = collection.query(query_embeddings=vec.astype(float).tolist(), n_results=5)
    documents = results['documents'][0][:]
    prices = [m['price'] for m in results['metadatas'][0][:]]
    return documents, prices

In [20]:
find_similars(test[0])

(['Title: Digitech DOD Carcosa Fuzz Distortion Pedal  \nCategory: Electronics  \nBrand: DigiTech  \nDescription: An analog fuzz‑distortion pedal delivering classic to modern tones with standout treble and mid‑range clarity.  \nDetails: Features true‑bypass switching, multiple control knobs (BEFORE, OUTPUT, AFTER, HI‑CUT), a DEMHE‑HALI toggle for low‑mid boost, and a 9‑V power supply.',
  'Title: EarthQuaker Devices Plumes Small Signal Shredder – Purple Sparkle  \nCategory: Guitar Pedals  \nBrand: EarthQuaker Devices  \nDescription: An all‑analog overdrive pedal that delivers three distinct clipping voices and expansive headroom for crystal‑clear, three‑dimensional tone.  \nDetails: Features Flexi‑Switch relay‑based bypass, 9\u202fV power input, and a finely tuned tone control for sculpting low, mid, and high frequencies.',
  'Title: Dunlop Rotovibe Pedal  \nCategory: Audio Effects  \nBrand: Dunlop  \nDescription: A versatile chorus/vibrato pedal that simulates rotating speaker sounds w

In [21]:
# We need to give some context to GPT-5.1 by selecting 5 products with similar descriptions

def make_context(similars, prices):
    message = "For context, here are some other items that might be similar to the item you need to estimate.\n\n"
    for similar, price in zip(similars, prices):
        message += f"Potentially related product:\n{similar}\nPrice is ${price:.2f}\n\n"
    return message

In [22]:
documents, prices = find_similars(test[0])
print(make_context(documents, prices))

For context, here are some other items that might be similar to the item you need to estimate.

Potentially related product:
Title: Digitech DOD Carcosa Fuzz Distortion Pedal  
Category: Electronics  
Brand: DigiTech  
Description: An analog fuzz‑distortion pedal delivering classic to modern tones with standout treble and mid‑range clarity.  
Details: Features true‑bypass switching, multiple control knobs (BEFORE, OUTPUT, AFTER, HI‑CUT), a DEMHE‑HALI toggle for low‑mid boost, and a 9‑V power supply.
Price is $159.99

Potentially related product:
Title: EarthQuaker Devices Plumes Small Signal Shredder – Purple Sparkle  
Category: Guitar Pedals  
Brand: EarthQuaker Devices  
Description: An all‑analog overdrive pedal that delivers three distinct clipping voices and expansive headroom for crystal‑clear, three‑dimensional tone.  
Details: Features Flexi‑Switch relay‑based bypass, 9 V power input, and a finely tuned tone control for sculpting low, mid, and high frequencies.
Price is $99.00


In [23]:
def messages_for(item, similars, prices):
    message = f"Estimate the price of this product. Respond with the price, no explanation\n\n{item.summary}\n\n"
    message += make_context(similars, prices)
    return [{"role": "user", "content": message}]

In [24]:
documents, prices = find_similars(test[0])
print(messages_for(test[0], documents, prices)[0]['content'])

Estimate the price of this product. Respond with the price, no explanation

Title: Excess V2 Distortion/Modulation Pedal  
Category: Music Pedals  
Brand: Old Blood Noise  
Description: A versatile pedal offering distortion and three modulation modes—delay, chorus, and harmonized fifths—with full control over signal routing and expression.  
Details: Features include separate gain, tone, and volume controls; time, depth, and volume per modulation; order switching, soft‑touch bypass, and expression jack for dynamic control.

For context, here are some other items that might be similar to the item you need to estimate.

Potentially related product:
Title: Digitech DOD Carcosa Fuzz Distortion Pedal  
Category: Electronics  
Brand: DigiTech  
Description: An analog fuzz‑distortion pedal delivering classic to modern tones with standout treble and mid‑range clarity.  
Details: Features true‑bypass switching, multiple control knobs (BEFORE, OUTPUT, AFTER, HI‑CUT), a DEMHE‑HALI toggle for low‑

In [25]:
# The function for gpt-5-mini

def gpt_5__1_rag(item):
    documents, prices = find_similars(item)
    response = completion(model="gpt-5.1", messages=messages_for(item, documents, prices), reasoning_effort="none", seed=42)
    return response.choices[0].message.content

In [26]:
# How much does our favorite distortion pedal cost?

test[0].price

219.0

In [27]:
# Let's do this!!

gpt_5__1_rag(test[0])

'$229.00'

In [28]:
evaluate(gpt_5__1_rag, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$10 $54 $5 $20 $30 $30 $24 $40 $4 $220 $63 $20 $0 $9 $29 $3 $11 $17 $50 $29 $11 $4 $55 $15 $133 $254 $106 $5 $111 $62 $35 $30 $30 $32 $25 $30 $30 $26 $24 $3 $155 $15 $12 $75 $80 $8 $32 $4 $75 $128 $23 $95 $125 $5 $27 $76 $9 $40 $88 $13 $86 $45 $41 $30 $20 $0 $20 $256 $15 $114 $18 $2 $20 $0 $10 $17 $4 $1 $3 $4 $0 $7 $11 $16 $13 $90 $78 $141 $10 $6 $3 $5 $1 $1 $2 $108 $2 $38 $95 $163 $10 $23 $13 $20 $0 $102 $15 $280 $11 $70 $25 $26 $2 $6 $54 $30 $10 $5 $144 $117 $9 $10 $20 $36 $19 $10 $1 $86 $19 $59 $0 $47 $5 $5 $35 $2 $15 $79 $11 $41 $4 $49 $23 $1 $4 $8 $5 $161 $5 $8 $4 $54 $27 $21 $21 $1 $41 $31 $10 $10 $10 $14 $5 $5 $230 $0 $181 $22 $10 $4 $3 $3 $170 $8 $17 $31 $3 $3 $26 $3 $75 $15 $205 $41 $17 $13 $53 $17 $10 $8 $5 $1 $5 $11 $0 $10 $19 $3 $16 $7 

In [29]:
import modal
Pricer = modal.Cls.from_name("pricer-service", "Pricer")
pricer = Pricer()

In [30]:
def specialist(item):
    return pricer.price.remote(item.summary)


In [31]:
def get_price(reply):
    reply = reply.replace("$", "").replace(",", "")
    match = re.search(r"[-+]?\d*\.\d+|\d+", reply)
    return float(match.group()) if match else 0

## Download the Neural Network weights from Week 6 into this directory

The file `deep_neural_network.pth` here:

https://drive.google.com/drive/folders/1uq5C9edPIZ1973dArZiEO-VE13F7m8MK?usp=drive_link

In [32]:
import torch
from agents.deep_neural_network import DeepNeuralNetworkInference

# --- monkey-patch torch.load ---
_original_torch_load = torch.load

def _torch_load_cpu(*args, **kwargs):
    kwargs.setdefault("map_location", "cpu")
    return _original_torch_load(*args, **kwargs)

torch.load = _torch_load_cpu
# --------------------------------

runner = DeepNeuralNetworkInference()
runner.setup()
runner.load("deep_neural_network.pth")

# przywracamy oryginalne torch.load
torch.load = _original_torch_load


def deep_neural_network(item):
    return runner.inference(item.summary)


In [33]:
def ensemble(item):
    price1 = get_price(gpt_5__1_rag(item))
    price2 = specialist(item)
    price3 = deep_neural_network(item)
    return price1 * 0.8 + price2 * 0.1 + price3 * 0.1


In [34]:
evaluate(ensemble, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$15 $56 $11 $4 $18 $50 $22 $30 $3 $165 $36 $3 $1 $7 $23 $2 $15 $17 $42 $8 $3 $6 $31 $4 $85 $241 $122 $2 $114 $60 $34 $22 $16 $33 $14 $6 $29 $26 $30 $12 $147 $4 $9 $52 $86 $8 $19 $1 $78 $101 $21 $87 $155 $10 $29 $62 $11 $54 $84 $14 $101 $44 $49 $41 $61 $4 $12 $270 $12 $100 $17 $3 $44 $6 $10 $15 $2 $1 $2 $2 $10 $6 $10 $25 $12 $75 $102 $117 $2 $5 $3 $5 $1 $2 $1 $92 $1 $29 $78 $191 $9 $13 $10 $7 $1 $97 $10 $278 $7 $66 $25 $20 $1 $13 $41 $29 $11 $1 $116 $104 $9 $19 $17 $35 $11 $0 $1 $73 $25 $58 $4 $34 $3 $5 $38 $1 $21 $62 $21 $36 $0 $27 $4 $9 $4 $9 $6 $181 $11 $8 $3 $53 $23 $24 $18 $20 $45 $25 $54 $10 $2 $14 $7 $1 $313 $0 $162 $21 $7 $5 $17 $2 $194 $2 $27 $25 $2 $1 $2 $0 $56 $12 $158 $32 $15 $13 $58 $13 $4 $5 $5 $1 $5 $12 $1 $2 $10 $1 $16 $8 

In [35]:
root = logging.getLogger()
root.setLevel(logging.INFO)

In [36]:
from agents.frontier_agent import FrontierAgent

agent = FrontierAgent(collection)
agent.price("Quadcast HyperX condenser mic, connects via usb-c to your computer for crystal clear audio")

INFO:root:[Frontier Agent] Initializing Frontier Agent
INFO:root:[Frontier Agent] Frontier Agent is setting up with OpenAI
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
INFO:root:[Frontier Agent] Frontier Agent is ready
INFO:root:[Frontier Agent] Frontier Agent is performing a RAG search of the Chroma datastore to find 5 similar products


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[Frontier Agent] Frontier Agent has found similar products
INFO:root:[Frontier Agent] Frontier Agent is about to call gpt-5.1 with context including 5 similar products
INFO:root:[Frontier Agent] Frontier Agent completed - predicting $109.99


109.99

In [37]:
agent.price("Shure MV7+ professional podcaster microphone with usb-c and XLR outputs")

INFO:root:[Frontier Agent] Frontier Agent is performing a RAG search of the Chroma datastore to find 5 similar products


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[Frontier Agent] Frontier Agent has found similar products
INFO:root:[Frontier Agent] Frontier Agent is about to call gpt-5.1 with context including 5 similar products
INFO:root:[Frontier Agent] Frontier Agent completed - predicting $279.00


279.0

In [38]:
import torch

_original_torch_load = torch.load

def _torch_load_cpu(*args, **kwargs):
    kwargs.setdefault("map_location", "cpu")
    return _original_torch_load(*args, **kwargs)

torch.load = _torch_load_cpu

from agents.neural_network_agent import NeuralNetworkAgent
agent = NeuralNetworkAgent()

INFO:root:[Neural Network Agent] Neural Network Agent is initializing
INFO:root:Neural Network is using cpu
INFO:root:[Neural Network Agent] Neural Network Agent is ready and weights are loaded


In [39]:
agent.price("Shure MV7+ professional podcaster microphone with usb-c and XLR outputs")

INFO:root:[Neural Network Agent] Neural Network Agent is starting a prediction
INFO:root:[Neural Network Agent] Neural Network Agent completed - predicting $162.39


162.39224243164062

In [40]:
from agents.ensemble_agent import EnsembleAgent
agent = EnsembleAgent(collection)

INFO:root:[Ensemble Agent] Initializing Ensemble Agent
INFO:root:[Specialist Agent] Specialist Agent is initializing - connecting to modal
INFO:root:[Frontier Agent] Initializing Frontier Agent
INFO:root:[Frontier Agent] Frontier Agent is setting up with OpenAI
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
INFO:root:[Frontier Agent] Frontier Agent is ready
INFO:root:[Neural Network Agent] Neural Network Agent is initializing
INFO:root:Neural Network is using cpu
INFO:root:[Neural Network Agent] Neural Network Agent is ready and weights are loaded
INFO:root:[Ensemble Agent] Ensemble Agent is ready


In [45]:
from litellm import completion

resp = completion(
    model="ollama/llama3.2:1b",
    messages=[{"role":"user","content":"Powiedz 'ok'."}]
)
print(resp.choices[0].message["content"])


14:32:25 - LiteLLM:INFO: utils.py:3421 - 
LiteLLM completion() model= llama3.2:1b; provider = ollama
INFO:LiteLLM:
LiteLLM completion() model= llama3.2:1b; provider = ollama
14:32:37 - LiteLLM:INFO: utils.py:1302 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler


Ok.


In [49]:
agent.preprocessor.model_name = "ollama/llama3.2:1b"


In [50]:
for attr in ["preprocessor", "specialist", "router", "judge"]:
    obj = getattr(agent, attr, None)
    if obj is not None and hasattr(obj, "model_name"):
        obj.model_name = "ollama/llama3.2:1b"


In [51]:
agent.price("Shure MV7+ professional podcaster microphone with usb-c and XLR outputs")

INFO:root:[Ensemble Agent] Running Ensemble Agent - preprocessing text
14:36:01 - LiteLLM:INFO: utils.py:3421 - 
LiteLLM completion() model= llama3.2:1b; provider = ollama
INFO:LiteLLM:
LiteLLM completion() model= llama3.2:1b; provider = ollama
14:36:17 - LiteLLM:INFO: utils.py:1302 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
INFO:root:[Ensemble Agent] Pre-processed text using ollama/llama3.2:1b
INFO:root:[Specialist Agent] Specialist Agent is calling remote fine-tuned model
INFO:root:[Specialist Agent] Specialist Agent completed - predicting $299.00
INFO:root:[Frontier Agent] Frontier Agent is performing a RAG search of the Chroma datastore to find 5 similar products


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[Frontier Agent] Frontier Agent has found similar products
INFO:root:[Frontier Agent] Frontier Agent is about to call gpt-5.1 with context including 5 similar products
INFO:root:[Frontier Agent] Frontier Agent completed - predicting $249.00
INFO:root:[Neural Network Agent] Neural Network Agent is starting a prediction
INFO:root:[Neural Network Agent] Neural Network Agent completed - predicting $66.31
INFO:root:[Ensemble Agent] Ensemble Agent complete - returning $235.73


235.73085784912112